In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import os
import zipfile

# Define paths
zip_path = '/content/drive/MyDrive/nature_12K.zip'
unzip_target_path = '/content/drive/MyDrive/nature_12K'

# Unzip only if the target folder does not exist
if not os.path.exists(unzip_target_path):
    print("Unzipping and storing in Google Drive...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('/content/drive/MyDrive/nature_12K')
    print("Unzipping complete and saved in Drive!")
else:
    print("Already unzipped in Drive. Skipping unzip.")

Already unzipped in Drive. Skipping unzip.


In [17]:
!pip install pytorch_lightning

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from pytorch_lightning.loggers import WandbLogger
import wandb
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
from torchvision.datasets import ImageFolder
from PIL import Image

In [19]:
# Define the configurable CNN model.
class ConfigurableCNN(pl.LightningModule):
    def __init__(self, config):
        super(ConfigurableCNN, self).__init__()
        self.save_hyperparameters()  # Saves config to checkpoint

        self.config = config

        # Map activation string to function
        activation_map = {
            'ReLU': nn.ReLU,
            'GELU': nn.GELU,
            'SiLU': nn.SiLU,
            'Mish': nn.Mish,
        }
        activation_fn = activation_map.get(config.get('activation', 'ReLU'), nn.ReLU)

        # Build 5 convolution blocks: Conv -> Activation -> (BatchNorm) -> MaxPool -> (Dropout)
        conv_layers = []
        in_channels = 3  # Assuming RGB images
        num_layers = config.get('num_conv_layers', 5)
        filters = config.get('filters', [32, 64, 128, 256, 512])
        kernel_sizes = config.get('kernel_sizes', [3] * num_layers)
        use_batchnorm = config.get('use_batchnorm', False)
        dropout_rate = config.get('dropout_rate', 0.0)

        for i in range(num_layers):
            out_channels = filters[i]
            kernel_size = kernel_sizes[i]
            padding = kernel_size // 2  # To maintain spatial dimensions before pooling
            conv = nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding)
            block = [conv, activation_fn()]
            if use_batchnorm:
                block.append(nn.BatchNorm2d(out_channels))
            # Append max pooling layer to reduce spatial dimensions by a factor of 2
            block.append(nn.MaxPool2d(2))
            if dropout_rate > 0:
                block.append(nn.Dropout2d(dropout_rate))
            conv_layers.extend(block)
            in_channels = out_channels

        self.conv = nn.Sequential(*conv_layers)

        # After 5 pooling operations, the spatial dimensions of a 128x128 image become:
        # 128 / (2^5) = 4 (assuming the dimensions divide evenly)
        final_spatial = 4
        fc_input_dim = filters[-1] * (final_spatial ** 2)

        # Define the fully connected part: Dense layer and final output layer with 10 classes.
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(fc_input_dim, config.get('dense_neurons', 256)),
            activation_fn(),
            nn.Dropout(config.get('dropout_rate_dense', 0.0)),
            nn.Linear(config.get('dense_neurons', 256), 10)
        )

        # Loss function and learning rate
        self.criterion = nn.CrossEntropyLoss()
        self.lr = config.get('lr', 1e-3)

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log("train_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log("train_acc", acc, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

In [20]:
train_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])
val_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

In [21]:
import time  # Add this import

# Load the dataset WITHOUT loading images into memory upfront
base_dir = "/content/drive/MyDrive/nature_12K/inaturalist_12K"
train_dir = os.path.join(base_dir, "train")

# Create dataset that only tracks file paths (doesn't load images)
full_train_dataset = ImageFolder(root=train_dir, transform=None, loader=lambda x: x)

print("Generating stratified split indices...")
start_time = time.time()

# Get class labels from directory structure
labels = [label for _, label in full_train_dataset.samples]

# Create split indices
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_indices, val_indices = next(sss.split(np.zeros(len(labels)), labels))

print(f"Split completed in {time.time() - start_time:.2f} seconds")

# Now create proper datasets with on-demand loading
class LazyLoadDataset(torch.utils.data.Dataset):
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform

    def __getitem__(self, index):
        path, label = self.samples[index]
        img = Image.open(path).convert('RGB')  # Requires PIL Image
        if self.transform:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.samples)

# Create datasets with lazy loading
train_samples = [full_train_dataset.samples[i] for i in train_indices]
val_samples = [full_train_dataset.samples[i] for i in val_indices]

train_dataset = LazyLoadDataset(train_samples, transform=train_transforms)
val_dataset = LazyLoadDataset(val_samples, transform=val_transforms)

Generating stratified split indices...
Split completed in 0.01 seconds


In [26]:
# Test dataset (not used anywhere)
test_dir = os.path.join(base_dir, "test")
test_dataset = ImageFolder(root=test_dir, transform=val_transforms)

In [27]:
# DataLoader optimizations
batch_size = 64
train_loader = DataLoader(
    train_dataset, batch_size=batch_size,
    shuffle=True, num_workers=2, pin_memory=True
)
val_loader = DataLoader(
    val_dataset, batch_size=batch_size,
    shuffle=False, num_workers=2, pin_memory=True
)

In [28]:
default_config = {
    "num_conv_layers": 5,
    "filters": [32, 64, 128, 256, 512],
    "kernel_sizes": [3, 3, 3, 3, 3],
    "activation": "ReLU",   # Options: ReLU, GELU, SiLU, Mish
    "use_batchnorm": False,
    "dropout_rate": 0.2,       # For the convolution blocks
    "dense_neurons": 256,
    "dropout_rate_dense": 0.3, # For the dense layer
    "lr": 1e-3,
    "max_epochs": 20         # Set max_epochs to 10 here
}

In [29]:
def train_model():
    # Initialize the wandb logger with default configuration.
    wandb_logger = WandbLogger(project="CNN_inaturalist_12K", config=default_config)

    # Retrieve the merged configuration (i.e. defaults overridden by sweep parameters, if any)
    config = dict(wandb_logger.experiment.config)

    # Construct the run name using the merged configuration.
    run_name = f"act-{config['activation']}_f-{config['filters'][0]}to{config['filters'][-1]}_lr-{config['lr']}"

    # Update the wandb run's name so that it reflects the sweep parameters.
    wandb.run.name = run_name

    # Use the max_epochs from the configuration; default to 10 if not provided.
    max_epochs = config.get("max_epochs", 20)

    model = ConfigurableCNN(config)

    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator='gpu',
        precision=16,  # Use mixed-precision (fp16) training.
        logger=wandb_logger,
        log_every_n_steps=10,
    )
    trainer.fit(model, train_loader, val_loader)


In [30]:
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'activation': {
            'values': ['ReLU', 'GELU', 'SiLU', 'Mish']
        },
        'filters': {
            'values': [
                [32, 64, 128, 256, 512],
                [32, 32, 64, 64, 128]
            ]
        },
        'use_batchnorm': {
            'values': [True, False]
        },
        'dropout_rate': {
            'values': [0.2, 0.3]
        },
        'dense_neurons': {
            'values': [256, 512]
        },
        'lr': {
            'values': [1e-3, 5e-4]
        },
        'dropout_rate_dense': {
            'values': [0.2, 0.3]
        },
        'max_epochs': {
            'values': [10, 20]
        },
        'batch_size': {
            'values': [32, 64]
        }
    }
}


In [31]:
# Create the sweep – this returns a unique sweep_id.
sweep_id = wandb.sweep(sweep_config, project="CNN_inaturalist_12K")
print("Sweep ID:", sweep_id)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: qurw00w9
Sweep URL: https://wandb.ai/mrsagarbiswas-iit-madras/CNN_inaturalist_12K/sweeps/qurw00w9
Sweep ID: qurw00w9


In [32]:
# Start the sweep agent; adjust count as desired (here count=1 for testing).
wandb.agent(sweep_id, train_model, count=5)

wandb: Agent Starting Run: qvpt23hs with config:
wandb: 	activation: Mish
wandb: 	batch_size: 64
wandb: 	dense_neurons: 256
wandb: 	dropout_rate: 0.3
wandb: 	dropout_rate_dense: 0.2
wandb: 	filters: [32, 64, 128, 256, 512]
wandb: 	lr: 0.0005
wandb: 	max_epochs: 10
wandb: 	use_batchnorm: True
wandb: Currently logged in as: mrsagarbiswas (mrsagarbiswas-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/lightning_fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | conv      | Sequential       | 1.6 M  | train
1 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
train_acc,▁▄▅▆▇▇▇▇██
train_loss,█▆▄▄▃▂▂▂▁▁
trainer/global_step,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
val_acc,▁▄▄▅▇▇▇███
val_loss,█▅▅▃▂▂▂▁▁▁
epoch,9
train_acc,0.30041
train_loss,1.96219
trainer/global_step,1249
val_acc,0.3205
